In [20]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

import yaml

from paths import *
from scripts import manipulation
from scripts import io

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
from scripts import vis_layout

from os import listdir
import glob


##  Portal da Transparência | Organizações Sociais de Saúde - OSS

 - [Portal da Transparência OSS](http://www.portaldatransparencia.saude.sp.gov.br/index.php)

In [94]:
geo_sp = gpd.read_file('../data/geodata/sp_municipios.json')

In [135]:
#oss qualificadas
quali = pd.concat(map(pd.read_csv, glob.glob('../data/oss_qualificadas/years/*.csv')))
quali_orgs = pd.concat(map(pd.read_csv, glob.glob('../data/oss_qualificadas/organizacoes/*.csv')))

#oss contratadas
contrat = pd.concat(map(pd.read_csv, glob.glob('../data/oss_contratualizadas/years/*.csv')))
contrat_org = pd.concat(map(pd.read_csv, glob.glob('../data/oss_contratualizadas/organizacoes/*.csv')))

# oss kpis
indicadores = pd.concat(map(pd.read_csv, glob.glob('../data/oss_indicadores/*.csv')))
indicadores = indicadores[indicadores['Unidade']!='Unidade']

#oss convocadas
convocacoes = pd.concat(map(pd.read_csv, glob.glob('../data/oss_convocacoes/*.csv')))

## meso region data
cols = ['geocodigo','nome_uf','nome_municipio','nome_mesorregiao']
sp = geo_sp[cols]
sp['Municipio'] = sp['nome_municipio']
sp['nome_municipio']          = sp['nome_municipio'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

#oss and cities data
municipios_quali = pd.concat(map(pd.read_csv, glob.glob('../data/oss_cidades/*.csv')))
municipios_quali = municipios_quali.reset_index(drop=True)
municipios_quali['municipio'] = municipios_quali['municipio'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
municipios_quali = municipios_quali.merge(sp, left_on='municipio', right_on='nome_municipio',how='left')

#oss with cities
oss = municipios_quali.sort_values(by='Ano', ascending=False).drop_duplicates(subset = ['oss','unidade'], keep='first').reset_index(drop=True).sort_values(by='oss', ascending=True).reset_index(drop=True)
cols = ['oss','unidade','cnes','cnpj','geocodigo','nome_uf','Municipio','endereco','bairro','nome_mesorregiao']
oss = oss[cols]

In [136]:
oss['nome_mesorregiao'].value_counts()

Metropolitana de São Paulo      78
Macro Metropolitana Paulista    11
São José do Rio Preto           10
Ribeirão Preto                   8
Bauru                            8
Vale do Paraíba Paulista         7
Litoral Sul Paulista             6
Presidente Prudente              4
Assis                            3
Itapetininga                     3
Campinas                         3
Araçatuba                        3
Araraquara                       2
Marília                          1
Name: nome_mesorregiao, dtype: int64

In [156]:
mask = oss['nome_mesorregiao']=='Vale do Paraíba Paulista'
oss[mask]

,oss,unidade,cnes,cnpj,geocodigo,nome_uf,Municipio,endereco,bairro,nome_mesorregiao
19,Associação Paulista para o Desenvolvimento da Medicina - SPDM,Ambulatório Médico de Especialidades São José dos Campos(AME São José dos Campos),6294049.0,61.699.567/0022-17,3549904,São Paulo,São José dos Campos,"Av. Engenheiro Francisco José Longo, 925",Jd. São Dimas,Vale do Paraíba Paulista
24,Associação Paulista para o Desenvolvimento da Medicina - SPDM,Centro de Medicina de Reabilitação Lucy Montoro de São José dos Campos(Lucy Montoro de São José dos Campos),6889549.0,61.699.567/0036-12,3549904,São Paulo,São José dos Campos,"Saigiro Nakamura, 600",Centro,Vale do Paraíba Paulista
96,Instituto Sócrates Guanaes - ISG,Hospital Regional de São José dos Campos(Hospital Regional de São José dos Campos),9491252.0,NaN,3549904,São Paulo,São José dos Campos,"Avenida Fortaleza, SN",Parque Industrial,Vale do Paraíba Paulista
100,Instituto Sócrates Guanaes - ISG,Ambulatório Médico de Especialidades São José dos Campos(AME São José dos Campos),6294049.0,61.699.567/0022-17,3549904,São Paulo,São José dos Campos,"Av. Engenheiro Francisco José Longo, 925",Jd. São Dimas,Vale do Paraíba Paulista
137,Serviço Social da Construção Civil do Estado de São Paulo - SECONCI,Ambulatório Médico de Especialidades Caraguatatuba (AME Caraguatatuba),6233848.0,46.543.781/0012-14,3510500,São Paulo,Caraguatatuba,"Av. Acre, 1081",Indaia,Vale do Paraíba Paulista
141,Serviço Social da Construção Civil do Estado de São Paulo - SECONCI,Ambulatório Médico de Especialidades de Lorena(AME Lorena),9030557.0,NaN,3527207,São Paulo,Lorena,"Rua Dom Bosco, 562",Centro,Vale do Paraíba Paulista
146,Sociedade Assistencial Bandeirantes,Ambulatório Médico de Especialidades Caraguatatuba (AME Caraguatatuba),6233848.0,46.543.781/0012-14,3510500,São Paulo,Caraguatatuba,"Av. Acre, 1081",Indaia,Vale do Paraíba Paulista


In [ ]:
# municipios_quali[mask].head()

In [187]:
municipios_quali

,Tipo,Número,Ano,Arquivo,oss,unidade,municipio,endereco,bairro,tipo,cnes,cnpj,geocodigo,nome_uf,nome_municipio,nome_mesorregiao,Municipio
0,Contrato de Gestão 2006,-,2006,Visualizar,Associação Congregação de Santa Catarina,Hospital Geral de Pedreira(Hospital Geral de Pedreira),Sao Paulo,"Rua João Francisco de Moura, 251",Vila Campo Grande,Hospitais,2066092.0,60.922.168/0021-20,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
1,Contrato de Gestão 2007,-,2007,Visualizar,Fundação do ABC,Hospital Estadual Mário Covas de Santo André(Hospital Estadual de Santo André),Santo Andre,"Rua Dr. Henrique Calderazzo, 321",Paraíso,Hospitais,2080273.0,57.571.275/0006-07,3547809,São Paulo,Santo Andre,Metropolitana de São Paulo,Santo André
2,Contrato de Gestão 2007,-,2007,Visualizar,Associação Lar São Francisco de Assis na Providência de Deus,Hospital Regional Porto Primavera(Hospital Regional Porto Primavera),Rosana,"Avenida dos Barrageiros, sem no - Quadra 36",Distrito de Rosana,Hospitais,2750546.0,53.221.255/0028-60,3544251,São Paulo,Rosana,Presidente Prudente,Rosana
3,Contrato de Gestão 2007,-,2007,Visualizar,Associação Congregação de Santa Catarina,Pólo de Atenção Intensiva em Saúde Mental da Zona Norte - PAI Zona Norte(PAI Zona Norte),Sao Paulo,"Rua Voluntários da Pátria, 4301",Mandaqui,Hospitais,2077574.0,60.922.168/0032-82,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
4,Contrato de Gestão 2007,-,2007,Visualizar,Casa de Saúde Santa Marcelina,Ambulatório Estadual de Especialidades Médicas da Zona Leste(AME Zona Leste),Sao Paulo,"Rua Maruins, 15",Itaquera,AME,NaN,60.742.616/0014-84,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262,Termo Aditivo 02/2020,2,2020,Visualizar,Serviço Social da Construção Civil do Estado de São Paulo - SECONCI,Serviço Estadual de Diagnóstico por Imagem II(SEDI II),Sao Paulo,"Avenida Almirante Delamare, 1534",Jardim Nova Heliópolis,Sedi,6926207.0,61.687.356/0030-74,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
2263,Contrato de Gestão 2020,-,2020,Visualizar,Fundação Instituto de Pesquisa e Estudo de Diagnóstico por Imagem - FIDI,Serviço Estadual de Diagnóstico por Imagem III(SEDI III),Sao Paulo,"Avenida Paulista, 1294 - 21o andar",Bela Vista,Sedi,6927726.0,55.401.178/0001-36,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
2264,Termo Aditivo 01/2020,1,2020,Visualizar,Fundação Instituto de Pesquisa e Estudo de Diagnóstico por Imagem - FIDI,Serviço Estadual de Diagnóstico por Imagem III(SEDI III),Sao Paulo,"Avenida Paulista, 1294 - 21o andar",Bela Vista,Sedi,6927726.0,55.401.178/0001-36,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo
2265,Termo Aditivo 01/2020 (Contrato de Gestão 2020),1,2020,Visualizar,Fundação Instituto de Pesquisa e Estudo de Diagnóstico por Imagem - FIDI,Serviço Estadual de Diagnóstico por Imagem III(SEDI III),Sao Paulo,"Avenida Paulista, 1294 - 21o andar",Bela Vista,Sedi,6927726.0,55.401.178/0001-36,3550308,São Paulo,Sao Paulo,Metropolitana de São Paulo,São Paulo


## Portal Financeiro do Gestor
 - [Portal Financeiro do Gestor](http://www.portalfinanceirodogestor.saude.sp.gov.br/)

In [174]:
convenios =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/convenio/*.csv')))

fundo =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/fundo/*.csv')))

proprio =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/proprios/*.csv')))


oss_valores =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/oss/ano/*.csv')))
# oss_valores['OSS'] = oss_valores['OSS'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [186]:
mask = oss_valores['ano'] == 2020
oss_valores[mask].sort_values(by=['Pago'], ascending=False).head(50)

,OSS,Gerenciadora atual,Contratado,Pago,ano
0,Hospital Santo André,Fundação ABC,NaN,1071.0,2020
1,Hospital Guarulhos,SPDM,NaN,973.0,2020
2,Hospital Luzia de Pinho Melo,SPDM,NaN,948.0,2020
3,Hospital Presidente Prudente,Lar São Francisco,NaN,928.0,2020
4,Hospital Bauru,FAMESP,NaN,924.0,2020
5,Hospital Brigadeiro,SPDM,NaN,840.0,2020
6,Hospital Grajau,Sirío Libanês,NaN,790.0,2020
7,Hospital Pedreira,SPDM,NaN,778.0,2020
8,Hospital Pirajussara,SPDM,NaN,754.0,2020
9,Conjunto Hospitalar de Sorocaba,SECONCI - SEDE,NaN,730.0,2020
